# Testing Environment Generation

This notebook can be used to generate the q_and_a_gen.json ai_answers that are required for comparing the Prompto Chatbot against target answers.

In [36]:
!pip install langchain openai


[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip


In [1]:
import json

In [20]:
from dotenv import load_dotenv
load_dotenv()

True

In [18]:
qa_json_file_path = "q_and_a.json"
qa_gen_json_file_path = "q_and_a_gen.json"

In [5]:
with open(qa_json_file_path, "r") as qa_json_file:
    qa_json = json.load(qa_json_file)

In [8]:
qa_data = qa_json["data"]

In [46]:
from langchain.prompts import PromptTemplate

template = "You are a helpful assistant for a prototyping framework called pronto that is build on top of the Godot Game Engine. When asked a question please stick to the provided information and don't deviate too much. Answer short and precise. If you don't know the answer, reply with 'I don't know'. With the information provided below please answer the following question.\nQuestion: {question}\nHere is the information to answer the question with:\n\n{retrieve}"

prompt = PromptTemplate(template=template, input_variables=["question", "retrieve"])

In [47]:
from langchain_community.chat_models import ChatOpenAI

llm = ChatOpenAI()

In [48]:
from langchain.chains import LLMChain

llm_chain = LLMChain(prompt=prompt, llm=llm)

In [49]:
for data in qa_data:
    question = data["question"]
    retrieve = "\n\n".join([f"Source: {entry['source']}\nContent: {entry['content']}" for entry in data["retrieve"]])
    data["ai_answer"] = llm_chain.run({
        "question": question,
        "retrieve": retrieve
    })


In [50]:
with open(qa_gen_json_file_path, "w") as qa_gen_json_file:
    qa_gen_json_file.write(json.dumps({"data":qa_data}))